In [22]:
import pandas as pd
import datetime
from datetime import date, time, timedelta
import itertools
import time
from numpy import average

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
# Global variables
BATTERY_POWER = 300
BATTERY_CAP = 580
CHARGE_EFF = 90
DISCHARGE_EFF = 90
MLF = 0.991

In [5]:
def raw_power(charge_forecast, discharge_forecast, opening_cap):
    if charge_forecast == 1 and discharge_forecast == 0:
        return -min(BATTERY_POWER, (BATTERY_CAP - opening_cap)/(CHARGE_EFF/100)*2)
    elif charge_forecast == 0 and discharge_forecast == 1:
        return min(BATTERY_POWER, (opening_cap/(DISCHARGE_EFF/100))*2 )
    else:
        return 0

In [6]:
def market_dispatch(raw_power):
    if raw_power < 0:
        return (raw_power/2)
    elif raw_power > 0:
        return (raw_power/2)*DISCHARGE_EFF/100
    else:
        return 0

In [7]:
def market_revenue(market_dispatch,spot_price):
    if market_dispatch < 0:
        return market_dispatch*spot_price*(1/MLF)
    elif market_dispatch > 0:
        return market_dispatch*spot_price*MLF
    else:
        return 0

In [8]:
def closing_capacity(market_dispatch, opening_cap):
    if market_dispatch < 0:
        x = opening_cap - market_dispatch * (CHARGE_EFF/100)
        return (max(0, min(round(x), BATTERY_CAP)))
    elif market_dispatch > 0:
        x = opening_cap - market_dispatch * (100/DISCHARGE_EFF)
        return (max(0, min(round(x), BATTERY_CAP)))
    else:
        x = opening_cap - market_dispatch * (100/DISCHARGE_EFF)
        return (max(0, min(round(x), BATTERY_CAP)))

In [27]:
def past_future_prices(index, subsample):
    '''Compute the average price of the past and future 10 periods relative to the current period'''
    past_avg = 0
    future_avg = 0
    
    if index < 10:   # if number of past prices less than 10
        past_avg = subsample.iloc[0:index]['Spot Price'].mean()
        future_avg = subsample.iloc[index+1:index + 11]['Spot Price'].mean()
    elif index >= (len(subsample) - 10):   # if number of future prices less than 10
        past_avg = subsample.iloc[index - 10:index]['Spot Price'].mean()
        future_avg = subsample.iloc[index+1:len(subsample)]['Spot Price'].mean()
    else:
        past_avg = subsample.iloc[index - 10:index]['Spot Price'].mean()
        future_avg = subsample.iloc[index+1:index + 11]['Spot Price'].mean()
        
    return past_avg, future_avg

In [ ]:
def past_future_prices(index, subsample):
    '''Compute the average price of the past and future 10 periods relative to the current period'''
    past_avg = 0
    future_avg = 0
    
    if (index < 10) and (index != 0):   # if number of past prices less than 10
        past_df = subsample.iloc[0:index]['Spot Price'].to_frame()
        past_df['Weights'] = list(range(1,index+1,1))
        past_avg = round(average(past_df['Spot Price'], weights = past_df['Weights']),2)
        
        future_df = subsample.iloc[index+1:index + 11]['Spot Price'].to_frame()
        future_df['Weights'] = list(range(10,0,-1))
        future_avg = round(average(future_df['Spot Price'], weights = future_df['Weights']),2)
        
    elif (index >= (len(subsample) - 10)) and (index != len(subsample) - 1):   # if number of future prices less than 10
        past_df = subsample.iloc[index - 10:index]['Spot Price'].to_frame()
        past_df['Weights'] = list(range(1,11,1))
        past_avg = round(average(past_df['Spot Price'], weights = past_df['Weights']),2)
        
        future_df = subsample.iloc[index+1:len(subsample)]['Spot Price'].to_frame()
        future_df['Weights'] = list(range(10,10-(len(subsample)-index)+1,-1))
        future_avg = round(average(future_df['Spot Price'], weights = future_df['Weights']),2)
        
    elif index == len(subsample) - 1:
        past_df = subsample.iloc[index - 10:index]['Spot Price'].to_frame()
        past_df['Weights'] = list(range(1,11,1))
        past_avg = round(average(past_df['Spot Price'], weights = past_df['Weights']),2)
        
        future_avg = subsample.iloc[index,]['Spot Price']
        
    elif index == 0:
        past_avg = subsample.iloc[index,]['Spot Price']
        
        future_df = subsample.iloc[index+1:index + 11]['Spot Price'].to_frame()
        future_df['Weights'] = list(range(10,0,-1))
        future_avg = round(average(future_df['Spot Price'], weights = future_df['Weights']),2)
        
    else:
        past_df = subsample.iloc[index - 10:index]['Spot Price'].to_frame()
        past_df['Weights'] = list(range(1,11,1))
        past_avg = round(average(past_df['Spot Price'], weights = past_df['Weights']),2)
        
        future_df = subsample.iloc[index+1:index + 11]['Spot Price'].to_frame()
        future_df['Weights'] = list(range(10,0,-1))
        future_avg = round(average(future_df['Spot Price'], weights = future_df['Weights']),2)
        
    return past_avg, future_avg

In [9]:
# Current > Past  --> Discharge
# Current < Future --> Charge

def charging_condition(index, current_price, past_price, future_price):
    '''Determine whether the current period is forecasted to charge or discharge'''
    
    comparison_threshold = 10
    current_past_diff = current_price - past_price
    current_future_diff = future_price - current_price
    
    # Discharge conditions
    if (current_price > past_price) and (current_price > future_price):
        if current_past_diff >= comparison_threshold:
            subsample["Discharge Forecast"].values[index] = 1
    
    # Charge conditions
    elif (current_price < past_price) and (current_price < future_price):
        if current_future_diff >= comparison_threshold:
            subsample["Charge Forecast"].values[index] = 1
    
    # Discharge/Charge when a conflict occurs
    elif (current_price > past_price) and (current_price < future_price):
        if current_past_diff >= comparison_threshold and current_future_diff >= comparison_threshold:
            if max(current_past_diff, current_future_diff) == current_past_diff:
                subsample["Discharge Forecast"].values[index] = 1
            else:
                subsample["Charge Forecast"].values[index] = 1
        elif current_past_diff >= comparison_threshold and current_future_diff < comparison_threshold:
            subsample["Discharge Forecast"].values[index] = 1
        elif current_future_diff >= comparison_threshold and current_past_diff < comparison_threshold:
            subsample["Charge Forecast"].values[index] = 1

In [10]:
df = pd.read_excel("../../data/market_data.xlsx") 

In [11]:
vic_spotprice = df.filter(items=['Time (UTC+10)', 'Regions VIC Trading Price ($/MWh)'])
vic_spotprice = vic_spotprice.rename(columns={'Regions VIC Trading Price ($/MWh)': 'Spot Price', 'Time (UTC+10)': 'Time'})

# Change column type to datetime type
vic_spotprice['Time'] = pd.to_datetime(vic_spotprice['Time'])

In [28]:
subsample = vic_spotprice.copy()

In [29]:
# Create new columns

subsample["Past Average"] = 0.0
subsample["Future Average"] = 0.0
subsample["Charge Forecast"] = 0
subsample["Discharge Forecast"] = 0
subsample["Raw Power"] = 0
subsample["Market Dispatch"] = 0
subsample["Market Revenue"] = 0
subsample["Opening Capacity"] = 0
subsample["Closing Capacity"] = 0

In [30]:
%%time

# Determine the charge and discharging behaviour of each period
for index, row in subsample.iterrows():
    past_average, future_average = past_future_prices(index, subsample)
    subsample["Past Average"].values[index] = past_average
    subsample["Future Average"].values[index] = future_average
    
    current_price = subsample.iloc[index]["Spot Price"]

    charging_condition(index, current_price, past_average, future_average)
    
    # Compute technical parameters of each period
    if index != 0:
        subsample["Opening Capacity"].values[index] = subsample["Closing Capacity"].values[index-1]
    
    subsample["Raw Power"].values[index] = raw_power(subsample["Charge Forecast"].values[index], subsample["Discharge Forecast"].values[index], subsample["Opening Capacity"].values[index])
    subsample["Market Dispatch"].values[index] = market_dispatch(subsample["Raw Power"].values[index])
    subsample["Market Revenue"].values[index] = market_revenue(subsample["Market Dispatch"].values[index], subsample["Spot Price"].values[index])
    subsample["Closing Capacity"].values[index] = closing_capacity(subsample["Market Dispatch"].values[index], subsample["Opening Capacity"].values[index])

CPU times: user 1min 58s, sys: 438 ms, total: 1min 58s
Wall time: 1min 58s


In [31]:
# Calculate the total market revenue
sum(subsample["Market Revenue"])

108771072

In [22]:
subsample.head(50)

,Time,Spot Price,Past Average,Future Average,Charge Forecast,Discharge Forecast,Raw Power,Market Dispatch,Market Revenue,Opening Capacity,Closing Capacity
0,2018-01-01 00:00:00,90.43,NaN,72.021,0,0,0,0,0,0,0
1,2018-01-01 00:30:00,92.46,90.430000,69.141,0,0,0,0,0,0,0
2,2018-01-01 01:00:00,87.62,91.445000,66.369,0,0,0,0,0,0,0
3,2018-01-01 01:30:00,73.08,90.170000,65.528,0,0,0,0,0,0,0
4,2018-01-01 02:00:00,70.18,85.897500,65.199,0,0,0,0,0,0,0
5,2018-01-01 02:30:00,67.43,82.754000,65.162,0,0,0,0,0,0,0
6,2018-01-01 03:00:00,66.31,80.200000,65.280,0,0,0,0,0,0,0
7,2018-01-01 03:30:00,67.72,78.215714,65.220,0,0,0,0,0,0,0
8,2018-01-01 04:00:00,65.50,76.903750,65.417,0,0,0,0,0,0,0
9,2018-01-01 04:30:00,64.50,75.636667,66.069,0,0,0,0,0,0,0
